**IMPLEMENTING MULTIPLE REINFORCEMENT AND DEEP REINFORCEMENT TECHNIQUES TO TRAIN OUR ALGORITHMIC TRADING MODEL**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv('/content/btc_1h.csv')

In [ ]:
df.head()

datetime      open      high       low     close      volume
0  2018-01-01 05:30:00  13715.65  13715.65  13400.01  13529.01  443.356199
1  2018-01-01 06:30:00  13528.99  13595.89  13155.38  13203.06  383.697006
2  2018-01-01 07:30:00  13203.00  13418.43  13200.00  13330.18  429.064572
3  2018-01-01 08:30:00  13330.26  13611.27  13290.00  13410.03  420.087030
4  2018-01-01 09:30:00  13434.98  13623.29  13322.15  13601.01  340.807329

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

**USING MINMAX SCALER TO PREPROCESS THE DATA**

In [ ]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[['Open', 'High', 'Close', 'Low', 'Volume']])
df_scaled = pd.DataFrame(df_scaled, columns=['Open', 'High', 'Close', 'Low', 'Volume'])


In [ ]:
state_dim = len(df.columns)
state_dim

7

In [ ]:
import matplotlib.pyplot as plt

**IMPLEMENTING Q-LEARNING**

In [ ]:
# Define Q-learning parameters
gamma = 0.95  # discount factor
alpha = 0.1   # learning rate
epsilon = 0.1  # exploration-exploitation trade-off

# Define the number of bins for discretization
num_bins = 5

# Discretize the state space
df_scaled_bins = pd.DataFrame()
for column in df_scaled.columns:
    df_scaled_bins[column] = pd.cut(df_scaled[column], bins=num_bins, labels=False)

# Initialize Q-network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(num_bins,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2)  # 2 actions: Buy or Sell
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

**TRAINING THE DEEP Q-LEARNING MODEL**

In [ ]:
# Training the DQN model
def train_dqn(df, model, gamma, alpha, epsilon, num_episodes=1):
    q_values = np.zeros((1, num_bins))
    for episode in range(num_episodes):
        state = np.array(df_scaled_bins.iloc[0])  # Initial state
        state = np.reshape(state, (1, num_bins))  # Reshape for the model input

        total_reward = 0

        for t in range(1, len(df_scaled)):
            # Epsilon-greedy policy
            if np.random.rand() < epsilon:
                action = np.random.choice([0, 1])
            else:
                q_values = model.predict(state)
                action = np.argmax(q_values)

            next_state = np.array(df_scaled_bins.iloc[t])
            next_state = np.reshape(next_state, (1, num_bins))

            reward = df_scaled['Close'].iloc[t] - df_scaled['Close'].iloc[t - 1]

            target = reward + gamma * np.max(model.predict(next_state))
            q_values[0, action] = (1 - alpha) * q_values[0, action] + alpha * target

            model.fit(state, q_values, epochs=1, verbose=0)

            state = next_state
            total_reward += reward

        print(f"Episode {episode + 1}, Total Reward: {total_reward}")

# Train the DQN model
train_dqn(df_scaled, model, gamma, alpha, epsilon, num_episodes=50)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 25ms/step
Episode 45, Total Reward: -0.3815214357376322
1/1 [==============================] - 0s 22ms/step
Episode 46, Total Reward: -0.3815214357376322
1/1 [==============================] - 0s 24ms/step
Episode 47, Total Reward: -0.3815214357376322
1/1 [==============================] - 0s 19ms/step
Episode 48, Total Reward: -0.3815214357376322
1/1 [==============================] - 0s 19ms/step
Episode 49, Total Reward: -0.3815214357376322
1/1 [==============================] - 0s 26ms/step
Episode 50, Total Reward: -0.3815214357376322


**TESTING THE TRAIN MODEL**

In [ ]:
# Testing the trained model
def test_dqn(model, df):
    state = np.array(df_scaled_bins.iloc[0])
    state = np.reshape(state, (1, num_bins))
    actions = []

    for t in range(1, len(df_scaled)):
        q_values = model.predict(state)
        action = np.argmax(q_values)
        actions.append(action)

        next_state = np.array(df_scaled_bins.iloc[t])
        next_state = np.reshape(next_state, (1, num_bins))

        state = next_state

    return actions

# Test the DQN model
actions_dqn = test_dqn(model, df_scaled)

# Add the predicted actions to your DataFrame
df['predicted_action_dqn'] = actions_dqn

# Visualize the results
plt.figure(figsize=(10, 6))
plt.plot(df['close'], label='Actual Close Price', color='blue')
plt.scatter(df.index, df['close'][df['predicted_action_dqn'] == 1], marker='^', color='green', label='Buy Signal')
plt.scatter(df.index, df['close'][df['predicted_action_dqn'] == 0], marker='v', color='red', label='Sell Signal')
plt.title('DQN Trading Strategy')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()

In [ ]:
!pip install gym


**RAINBOW DEEP Q-LEARNING**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import gym
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber

# Replace this with your actual DataFrame loading code
# df = pd.read_csv('your_data.csv')

# Define a simple environment
class CustomEnv(gym.Env):
    def __init__(self, data):
        self.data = data
        self.current_step = 0
        self.max_steps = len(data) - 1  # Assumes one episode corresponds to the entire dataset
        self.action_space = gym.spaces.Discrete(2)  # Replace with your actual action space
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(len(data.columns),), dtype=np.float32)  # Replace with your actual observation space

    def reset(self):
        self.current_step = 0
        return self.data.iloc[self.current_step].values

    def step(self, action):
        self.current_step += 1
        done = self.current_step == self.max_steps
        obs = self.data.iloc[self.current_step].values
        reward = 1.0 if action == 1 else 0.0  # Replace with your actual reward function
        return obs, reward, done, {}

# Build the Rainbow DQN model
def build_model(input_shape, num_actions):
    model = Sequential([
        Dense(32, activation='relu', input_shape=(input_shape,)),
        Dense(32, activation='relu'),
        Dense(num_actions, activation='linear')
    ])
    return model

# Implement Rainbow DQN agent
class RainbowDQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)  # Replay memory
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration-exploitation trade-off
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = build_model(state_size, action_size)
        self.target_model = build_model(state_size, action_size)
        self.target_model.set_weights(self.model.get_weights())
        self.optimizer = Adam(learning_rate=self.learning_rate, clipnorm=1.0)
        self.loss_fn = Huber()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def choose_action(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        q_values = self.model.predict(state.reshape(1, -1))
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        minibatch = np.array(self.memory)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
              target = (reward + self.gamma * np.amax(self.target_model.predict(np.array(next_state).reshape(1, -1))[0]))
            target_f = self.model.predict(state.reshape(1, -1))
            target_f[0][action] = target
            self.model.fit(state.reshape(1, -1), target_f, epochs=1, verbose=0)

    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i]
        self.target_model.set_weights(target_weights)

    def train(self, env, episodes, batch_size):
        for episode in range(episodes):
            state = env.reset()
            state = np.reshape(state, [1, env.observation_space.shape[0]])
            total_reward = 0
            for time in range(env.max_steps):
                action = self.choose_action(state)
                next_state, reward, done, _ = env.step(action)
                next_state = np.reshape(next_state, [1, env.observation_space.shape[0]])
                self.remember(state, action, reward, next_state, done)
                state = next_state
                total_reward += reward
                if done:
                    break
                if len(self.memory) > batch_size:
                    self.replay(batch_size)
                    self.target_train()
            print(f"Episode: {episode + 1}, Total Reward: {total_reward}")
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay

env = CustomEnv(df)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

agent = RainbowDQNAgent(state_size, action_size)
agent.train(env, episodes=1, batch_size=32)


**DEEP DETERMINISTIC POLICY GRADIENT**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.optimizers import Adam
import gym

# Define the actor and critic neural networks
def build_actor(state_dim, action_dim, action_bound):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(state_dim,)),
        Dense(32, activation='relu'),
        Dense(action_dim, activation='tanh')  # Tanh activation for bounded action space
    ])
    return model

def build_critic(state_dim, action_dim):
    state_input = Input(shape=(state_dim,))
    action_input = Input(shape=(action_dim,))
    x = Concatenate()([state_input, action_input])
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(1, activation='linear')(x)
    model = Model(inputs=[state_input, action_input], outputs=output)
    return model

# Define the DDPG agent
class DDPGAgent:
    def __init__(self, state_dim, action_dim, action_bound, tau=0.001, discount_factor=0.99):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.action_bound = action_bound
        self.tau = tau
        self.discount_factor = discount_factor

        # Initialize actor and critic networks
        self.actor = build_actor(state_dim, action_dim, action_bound)
        self.target_actor = build_actor(state_dim, action_dim, action_bound)
        self.critic = build_critic(state_dim, action_dim)
        self.target_critic = build_critic(state_dim, action_dim)

        # Initialize target networks with the same weights as the online networks
        self.target_actor.set_weights(self.actor.get_weights())
        self.target_critic.set_weights(self.critic.get_weights())

        # Set up optimizer for both actor and critic
        self.actor_optimizer = Adam(learning_rate=0.0001)
        self.critic_optimizer = Adam(learning_rate=0.001)

    def get_action(self, state):
        action = self.actor.predict(state.reshape(1, self.state_dim))[0]
        return action + np.random.normal(0, 0.1, self.action_dim)  # Add noise for exploration

    def train(self, states, actions, rewards, next_states, dones):
        # Train critic
        target_actions = self.target_actor.predict(next_states)
        target_q_values = self.target_critic.predict([next_states, target_actions])
        target_q_values = rewards + self.discount_factor * (1 - dones) * target_q_values
        self.critic_optimizer.minimize(lambda: self.critic_loss(states, actions, target_q_values))

        # Train actor
        actions_for_grad = self.actor(states)
        q_values_for_grad = self.critic([states, actions_for_grad])
        actor_gradients = tf.gradients(q_values_for_grad, actions_for_grad)
        self.actor_optimizer.minimize(lambda: -self.actor(states), var_list=self.actor.trainable_variables)

        # Update target networks
        self.update_target_networks()

    def critic_loss(self, states, actions, target_q_values):
        current_q_values = self.critic([states, actions])
        return tf.reduce_mean(tf.square(current_q_values - target_q_values))

    def update_target_networks(self):
        actor_weights = self.actor.get_weights()
        critic_weights = self.critic.get_weights()
        target_actor_weights = self.target_actor.get_weights()
        target_critic_weights = self.target_critic.get_weights()

        for i in range(len(actor_weights)):
            target_actor_weights[i] = self.tau * actor_weights[i] + (1 - self.tau) * target_actor_weights[i]

        for i in range(len(critic_weights)):
            target_critic_weights[i] = self.tau * critic_weights[i] + (1 - self.tau) * target_critic_weights[i]

        self.target_actor.set_weights(target_actor_weights)
        self.target_critic.set_weights(target_critic_weights)

# Example usage with a gym environment

###   USING GYM ENVIRONMENT  ###

env = gym.make('Pendulum-v0')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_bound = env.action_space.high

agent = DDPGAgent(state_dim, action_dim, action_bound)

num_episodes = 1000
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0

    for t in range(200):  # Assuming a maximum of 200 steps per episode
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        agent.train(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        if done:
            break

    print(f"Episode: {episode + 1}, Total Reward: {total_reward}")


**PROXIMAL POLICY OPTIMIZATION**




In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import gym

# Define the Actor-Critic neural network
def build_actor_critic(state_dim, action_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(state_dim,)),
        Dense(32, activation='relu'),
        Dense(action_dim, activation='softmax')  # Softmax for discrete action space
    ])

    actor = Sequential([
        model,
        Dense(action_dim, activation='linear', use_bias=False)
    ])

    critic = Sequential([
        model,
        Dense(1, activation='linear')
    ])

    return actor, critic

# Define the Proximal Policy Optimization (PPO) agent
class PPOAgent:
    def __init__(self, state_dim, action_dim, action_bound, gamma=0.99, epsilon=0.2, learning_rate=0.001):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon
        self.learning_rate = learning_rate

        self.actor, self.critic = build_actor_critic(state_dim, action_dim)

        self.optimizer_actor = Adam(learning_rate=learning_rate)
        self.optimizer_critic = Adam(learning_rate=learning_rate)

    def get_action(self, state):
        action_probs = self.actor.predict(state.reshape(1, self.state_dim))[0]
        action = np.random.choice(self.action_dim, p=action_probs)
        return action

    def train(self, states, actions, rewards, next_states, dones, old_probs):
        advantages, target_values = self.calculate_advantages_and_targets(states, rewards, next_states, dones)

        # Train actor
        actor_loss = self.train_actor(states, actions, advantages, old_probs)

        # Train critic
        critic_loss = self.train_critic(states, target_values)

        return actor_loss, critic_loss

    def train_actor(self, states, actions, advantages, old_probs):
        with tf.GradientTape() as tape:
            action_probs = self.actor(states)
            chosen_probs = tf.reduce_sum(tf.one_hot(actions, self.action_dim) * action_probs, axis=1)
            ratio = tf.exp(tf.math.log(chosen_probs) - tf.math.log(old_probs))
            surr1 = ratio * advantages
            surr2 = tf.clip_by_value(ratio, 1 - self.epsilon, 1 + self.epsilon) * advantages
            actor_loss = -tf.reduce_mean(tf.minimum(surr1, surr2))

        actor_gradients = tape.gradient(actor_loss, self.actor.trainable_variables)
        self.optimizer_actor.apply_gradients(zip(actor_gradients, self.actor.trainable_variables))

        return actor_loss.numpy()

    def train_critic(self, states, target_values):
        with tf.GradientTape() as tape:
            predicted_values = self.critic(states)
            critic_loss = tf.reduce_mean(tf.square(target_values - predicted_values))

        critic_gradients = tape.gradient(critic_loss, self.critic.trainable_variables)
        self.optimizer_critic.apply_gradients(zip(critic_gradients, self.critic.trainable_variables))

        return critic_loss.numpy()

    def calculate_advantages_and_targets(self, states, rewards, next_states, dones):
        returns = []
        advantages = []
        target_values = []

        discounted_return = 0
        for t in reversed(range(len(states))):
            discounted_return = rewards[t] + self.gamma * discounted_return * (1 - dones[t])
            value = self.critic(states[t])[0].numpy()
            advantages.append(discounted_return - value)
            returns.append(discounted_return)
            target_values.append(discounted_return)

        advantages = np.flip(advantages)
        returns = np.flip(returns)

        return advantages, np.array(target_values)

# Example usage with a gym environment
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

agent = PPOAgent(state_dim, action_dim)

num_episodes = 1000
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0

    states, actions, rewards, next_states, dones, old_probs = [], [], [], [], [], []

    while True:
        action_probs = agent.actor.predict(state.reshape(1, state_dim))[0]
        action = np.random.choice(action_dim, p=action_probs)
        next_state, reward, done, _ = env.step(action)

        states.append(state)
        actions.append(action)
        rewards.append(reward)
        next_states.append(next_state)
        dones.append(int(done))
        old_probs.append(action_probs[action])

        state = next_state
        total_reward += reward

        if done:
            break

    states = np.array(states)
    actions = np.array(actions)
    rewards = np.array(rewards)
    next_states = np.array(next_states)
    dones = np.array(dones)
    old_probs = np.array(old_probs)

    advantages, target_values = agent.calculate_advantages_and_targets(states, rewards, next_states, dones)

    actor_loss, critic_loss = agent.train(states, actions, rewards, next_states, dones, old_probs)

    print(f"Episode: {episode + 1}, Total Reward: {total_reward}, Actor Loss: {actor_loss}, Critic Loss: {critic_loss}")


**RECURRENT POLICY GRADIENT**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
import gym

# Define the Recurrent Policy Gradient (RPG) agent
class RPGLSTM:
    def __init__(self, state_dim, action_dim, hidden_dim=32, learning_rate=0.001):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.hidden_dim = hidden_dim
        self.learning_rate = learning_rate

        self.model = self.build_model()
        self.optimizer = Adam(learning_rate=learning_rate)

    def build_model(self):
        model = Sequential([
            LSTM(self.hidden_dim, input_shape=(None, self.state_dim)),
            Dense(self.action_dim, activation='softmax')
        ])
        return model

    def get_action(self, state_sequence):
        state_sequence = np.expand_dims(state_sequence, axis=0)
        action_probs = self.model.predict(state_sequence)[0]
        action = np.random.choice(self.action_dim, p=action_probs)
        return action

    def train(self, state_sequences, actions, rewards):
        with tf.GradientTape() as tape:
            action_probs = self.model(state_sequences)
            selected_action_probs = tf.reduce_sum(tf.one_hot(actions, self.action_dim) * action_probs, axis=-1)
            loss = -tf.reduce_sum(tf.math.log(selected_action_probs) * rewards)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

# Example usage with a gym environment

###      UTILIZING GYM ENVIRONMENT   ###

env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

agent = RPGLSTM(state_dim, action_dim)

num_episodes = 1000
for episode in range(num_episodes):
    state_sequence, actions, rewards = [], [], []
    state = env.reset()

    while True:
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)

        state_sequence.append(state)
        actions.append(action)
        rewards.append(reward)

        state = next_state

        if done:
            break

    state_sequence = np.array(state_sequence)
    actions = np.array(actions)
    rewards = np.array(rewards)

    agent.train(state_sequence, actions, rewards)

    total_reward = sum(rewards)
    print(f"Episode: {episode + 1}, Total Reward: {total_reward}")

# Note: This is a basic example, and you might need to adapt the code to your specific environment and problem. The model architecture and hyperparameters may need adjustment based on the characteristics of your data and task.


DUE TO LESS COMPUTATIONAL POWER WE COULD NOT RUN ALL THE REINFORCEMENT LEARNING CODES, BUT WE LOOKED AFTER THE LOGIC AND ALL THE MODEL TRAINING WHICH CAN BE IMPLEMENTED